In [7]:
pip install paho-mqtt==1.6.1

SyntaxError: invalid syntax (2042074864.py, line 1)

In [ ]:
"""Publish terminal messages to topic"""

import paho.mqtt.client as mqtt

# MQTT settings
broker_url = "3.138.185.79"
broker_port = 1883
username = "auca"
password = "gishushu"
topic = "auca_class"
client_id = "my_mqtt_client"

# Callback when the client receives a CONNACK response from the server
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully to broker")
    else:
        print(f"Failed to connect, return code {rc}\n")
        # If the client fails to connect then we should stop the loop
        client.loop_stop()

# Create a new instance of the MQTT client with a specific client ID
client = mqtt.Client(client_id, clean_session=True)
client.on_connect = on_connect  # attach the callback function to the client
client.username_pw_set(username, password)  # set username and password

client.connect(broker_url, broker_port, 60)  # connect to the broker

# Start the network loop in a separate thread
client.loop_start()

try:
    while True:
        message = input("Enter message to publish or type 'exit' to quit: ")
        if message.lower() == 'exit':
            break
        client.publish(topic, message, qos=2)
except KeyboardInterrupt:
    print("Program interrupted by user, exiting...")

# Stop the network loop and disconnect
client.loop_stop()
client.disconnect()


Connected successfully to broker
Enter message to publish or type 'exit' to quit: ab
Enter message to publish or type 'exit' to quit: cd


In [ ]:
"""Subscribe to messages on topic"""


import paho.mqtt.client as mqtt

# MQTT settings
broker_url = "3.138.185.79"
broker_port = 1883
username = "auca"
password = "gishushu"
topic = "auca_class"
client_id = "my_mqtt_client_subscriber"

# Callback when the client receives a CONNACK response from the server
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully to broker")
        # Subscribe to the topic once connected
        client.subscribe(topic, qos=2)
    else:
        print(f"Failed to connect, return code {rc}\n")

# Callback for when a PUBLISH message is received from the server
def on_message(client, userdata, msg):
    print(f"Message received on topic {msg.topic}: {msg.payload.decode()}")

# Create a new instance of the MQTT client with a specific client ID
client = mqtt.Client(client_id, clean_session=True)
client.on_connect = on_connect  # attach the connection callback function to the client
client.on_message = on_message  # attach the message callback function to the client

client.username_pw_set(username, password)  # set username and password
client.connect(broker_url, broker_port, 60)  # connect to the broker

# Start the network loop in a separate thread
client.loop_forever()


CREATE TABLE sensor (
   time        TIMESTAMPTZ       NOT NULL,
   location    TEXT              NOT NULL,
   device      TEXT              NOT NULL,
   voltage     DOUBLE PRECISION  NOT NULL,
   current     DOUBLE PRECISION  NOT NULL,
   frequency   DOUBLE PRECISION  NOT NULL,
   power       DOUBLE PRECISION  NOT NULL,
   energy      DOUBLE PRECISION  NOT NULL
);


## Create a standard PostgreSQL table:

CREATE TABLE sensor (
   time        TIMESTAMPTZ       NOT NULL,
   location    TEXT              NOT NULL,
   device      TEXT              NOT NULL,
   voltage     DOUBLE PRECISION  NOT NULL,
   current     DOUBLE PRECISION  NOT NULL,
   frequency   DOUBLE PRECISION  NOT NULL,
   power       DOUBLE PRECISION  NOT NULL,
   energy      DOUBLE PRECISION  NOT NULL,
   PRIMARY KEY (time, location, device)
);


## Convert the table to a hypertable. Specify the name of the table you want to convert, and the column that holds its time values.

SELECT create_hypertable('sensor', 'time', chunk_time_interval => interval '1 week');



## Insert Single Row

INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy) 
VALUES 
('2023-04-26 12:30:00+00', 'Factory A', 'Sensor 123', 230.5, 5.5, 60, 1267.5, 15000);


## Insert Multiple Rows

INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy) 
VALUES 
('2023-04-26 12:30:00+00', 'Factory A', 'Sensor 123', 230.5, 5.5, 60, 1267.5, 15000),
('2023-04-26 12:35:00+00', 'Factory B', 'Sensor 124', 220.0, 6.0, 50, 1320.0, 16000),
('2023-04-26 12:40:00+00', 'Factory C', 'Sensor 125', 240.0, 4.8, 50, 1152.0, 14000);

## Enable Compression

ALTER TABLE sensor SET (timescaledb.compress, timescaledb.compress_orderby = 'time');

## Add Compression Policy
SELECT add_compression_policy('sensor', INTERVAL '7 days');

## Show all chunks older than x minutes
SELECT show_chunks('sensor', older_than => INTERVAL '3 minutes');

## Compress all chunks older than 3 minutes
SELECT compress_chunk(i)
FROM show_chunks('sensor', older_than => INTERVAL '3 minutes') AS i;

## Drop Table
DROP TABLE TABLE_NAME;
DROP TABLE sensor;

## Enable Compression
ALTER TABLE sensor SET (timescaledb.compress, timescaledb.compress_orderby = 'time');

## Add Compression Policy
SELECT add_compression_policy('sensor', INTERVAL '7 days');

## Show all chunks older than x minutes
SELECT show_chunks('sensor', older_than => INTERVAL '3 minutes');

## Compress all chunks older than 3 minutes¶
SELECT compress_chunk(i) FROM show_chunks('sensor', older_than => INTERVAL '3 minutes') AS i;

## Drop Table
DROP TABLE TABLE_NAME; 
DROP TABLE sensor;

## Disk Size of a hypertable; both compressed and uncompressed chunks
SELECT hypertable_size('sensor');

## Retrieves the name and size of each hypertable present in the database
SELECT sensor, hypertable_size(format('%I.%I', hypertable_schema, hypertable_name)::regclass) FROM timescaledb_information.hypertables;

## Detailed view of the disk space usage of a hypertable; If running on a distributed hypertable, ordering by node_name would show the size distribution across different data nodes 
SELECT * FROM hypertable_detailed_size('sensor') ORDER BY node_name;

## Manually compresses a specific chunk identified by its internal name.
SELECT compress_chunk( '_timescaledb_internal._hyper_1_1_chunk');

## Provides compression statistics for all chunks of the specified hypertable.
SELECT * FROM chunk_compression_stats('sensor');

## Attempts to compress chunks of the 'sensor' hypertable that were created between three weeks ago and one week ago.
SELECT compress_chunk(i) from show_chunks('sensor', now() - interval '1 week', now() - interval '3 weeks') i;


## Use the psycopg2 library to interact with your PostgreSQL (TimescaleDB) database

pip install psycopg2

In [ ]:
"""
Insert 200k random data for 10 sensors

Explanation:
Fixed Time: All data points in this modified version share the same time value for the entire batch of 200,000 
inserts. This time is set just before the loop starts.
"""

import psycopg2
import random
from datetime import datetime, timedelta

# Database connection parameters
conn_params = {
    "dbname": 'your_database_name',
    "user": 'your_username',
    "password": 'your_password',
    "host": 'your_host_address'
}

# Function to generate random data, now with a fixed time parameter
def generate_random_data(sensor_id, fixed_time):
    return {
        "time": fixed_time,
        "location": f"Location {sensor_id}",
        "device": f"Sensor {sensor_id}",
        "voltage": random.uniform(220, 240),  # random voltage between 220 and 240
        "current": random.uniform(10, 20),    # random current between 10 and 20
        "frequency": random.uniform(50, 60),  # random frequency between 50 and 60
        "power": random.uniform(1000, 5000),  # random power between 1000 and 5000
        "energy": random.uniform(500, 1500)   # random energy between 500 and 1500
    }

# Connect to the PostgreSQL database
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

# Insert data in a loop
try:
    fixed_time = datetime.now()  # Capture a single moment in time for all entries
    for i in range(200000):  # Total of 200,000 insertions
        sensor_id = random.randint(1, 10)  # Randomly choose one of 10 sensors
        data = generate_random_data(sensor_id, fixed_time)
        
        # SQL insertion query
        query = """
        INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.execute(query, (data['time'], data['location'], data['device'], data['voltage'],
                               data['current'], data['frequency'], data['power'], data['energy']))
        
        # Commit every 1000 inserts to avoid too large transactions
        if i % 1000 == 0:
            conn.commit()
            print(f"Inserted {i+1} records")

    # Commit any remaining inserts
    conn.commit()
    print("Data insertion complete.")

except Exception as e:
    print("An error occurred:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()
